In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 20:36:29 WARN Utils: Your hostname, silverlining.local resolves to a loopback address: 127.0.0.1; using 192.168.0.224 instead (on interface en0)
24/03/02 20:36:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 20:36:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
spark.version

'3.5.1'

In [4]:
df_fhv = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [5]:
import pandas as pd

In [6]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [7]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [8]:
df_fhv_pd = pd.read_csv('fhv_tripdata_2019-10.csv.gz')

In [9]:
df_fhv_pd

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014
...,...,...,...,...,...,...,...
1897488,B03160,2019-10-31 23:38:00,2019-10-31 23:48:00,242.0,81.0,NaN,B02887
1897489,B03160,2019-10-31 23:11:00,2019-10-31 23:43:00,161.0,28.0,NaN,B02883
1897490,B03160,2019-10-31 23:13:00,2019-10-31 23:41:00,168.0,215.0,NaN,B02883
1897491,B03186,2019-10-31 23:02:32,2019-10-31 23:09:53,264.0,119.0,NaN,B03186


In [10]:
df_fhv_pd.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [11]:
spark.createDataFrame(df_fhv_pd)

DataFrame[dispatching_base_num: string, pickup_datetime: string, dropOff_datetime: string, PUlocationID: double, DOlocationID: double, SR_Flag: double, Affiliated_base_number: string]

In [12]:
df_fhv_pd.iteritems = df_fhv_pd.items

In [13]:
spark.createDataFrame(df_fhv_pd).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [15]:
from pyspark.sql import types

In [16]:
fhv_schema = types.StructType([
types.StructField('dispatching_base_num', types.StringType(), True), 
types.StructField('pickup_datetime', types.TimestampType(), True), 
types.StructField('dropOff_datetime', types.TimestampType(), True), 
types.StructField('PUlocationID', types.IntegerType(), True), 
types.StructField('DOlocationID', types.IntegerType(), True),
types.StructField('SR_Flag', types.DoubleType(), True), 
types.StructField('Affiliated_base_number', types.StringType(), True)])

In [17]:
df_fhv = spark.read \
    .option("header", "true") \
    .schema(fhv_schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [18]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [21]:
df_fhv \
    .repartition(6) \
    .write \
    .parquet('data/pq/fhv/2019/10') \
    .mode('overwrite')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/barbaraatkins/dev/projects/data-engineering-zoomcamp/05-batch/code/data/pq/fhv/2019/10 already exists. Set mode as "overwrite" to overwrite the existing path.

In [22]:
df_fhv = spark.read.parquet('data/pq/fhv/2019/10/*')

In [23]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [25]:
df_fhv.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [28]:
columns = df_fhv.columns

In [30]:
df_fhv.select(columns).show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02594|2019-10-19 12:32:00|2019-10-19 13:03:00|         157|         133|   NULL|                B02594|
|              B00647|2019-10-07 19:26:49|2019-10-07 19:42:43|         264|          78|   NULL|                B00647|
|              B01145|2019-10-20 02:30:04|2019-10-20 02:37:54|         264|         244|   NULL|                B01145|
|              B03060|2019-10-13 18:58:21|2019-10-13 19:11:24|         264|         123|   NULL|                B02888|
|              B02418|2019-10-07 21:40:00|2019-10-07 22:22:00|         264|         264|   NULL|                B00280|
|              B02243|2019-10-18 11:08:0

In [33]:
df_fhv.registerTempTable('fhv_trips_data')

In [43]:
spark.sql('''
          SELECT count(*) FROM fhv_trips_data WHERE DATE(pickup_datetime) = '2019-10-15'
          ''').show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [47]:
spark.sql('''
          SELECT pickup_datetime, dropoff_datetime, (unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime))/(3600) FROM fhv_trips_data
          ORDER BY 3 desc
          
          ''').show()

+-------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------+
|    pickup_datetime|   dropoff_datetime|((unix_timestamp(dropoff_datetime, yyyy-MM-dd HH:mm:ss) - unix_timestamp(pickup_datetime, yyyy-MM-dd HH:mm:ss)) / 3600)|
+-------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------+
|2019-10-11 18:00:00|2091-10-11 18:30:00|                                                                                                               631152.5|
|2019-10-28 09:00:00|2091-10-28 09:30:00|                                                                                                               631152.5|
|2019-10-31 23:46:33|2029-11-01 00:13:00|                                                                                                      87672.44083333333|
|2019-10-01 21:43:42|2027-10

In [48]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [50]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [51]:
df_zones.registerTempTable('zone_data')

In [52]:
spark.sql('''
          SELECT count(*) FROM zone_data
          ''').show()

+--------+
|count(1)|
+--------+
|     265|
+--------+



In [55]:
spark.sql('''
          SELECT count(*), Zone from fhv_trips_data fhv, zone_data zd 
          WHERE fhv.PUlocationID = zd.LocationID
          GROUP BY Zone
          ORDER BY 1 
          ''').show()

+--------+--------------------+
|count(1)|                Zone|
+--------+--------------------+
|       1|         Jamaica Bay|
|       2|Governor's Island...|
|       5| Green-Wood Cemetery|
|       8|       Broad Channel|
|      14|     Highbridge Park|
|      15|        Battery Park|
|      23|Saint Michaels Ce...|
|      25|Breezy Point/Fort...|
|      26|Marine Park/Floyd...|
|      29|        Astoria Park|
|      39|    Inwood Hill Park|
|      47|       Willets Point|
|      53|Forest Park/Highl...|
|      57|  Brooklyn Navy Yard|
|      62|        Crotona Park|
|      77|        Country Club|
|      89|     Freshkills Park|
|      98|       Prospect Park|
|     105|     Columbia Street|
|     110|  South Williamsburg|
+--------+--------------------+
only showing top 20 rows

